# The k-NN Algorithm
The k-nearest neighbours classifier/regressor is one of the simplest machine learning algorithms.

For example, when the algorithm wants to classify a new instance, it will look at all of the data it has and picks the 5 closest instances and use that to determine the class. This means there is no real training phase - it always uses all of the "training" data for each input.

## The data

We will take a look at a famous small dataset of three distinct species of Iris flowers:

| Iris Setosa | Iris Veriscolor | Iris Virginica |
|-------------|-----------------|----------------|
| <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg/450px-Kosaciec_szczecinkowaty_Iris_setosa.jpg" /> | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/Blue_Flag%2C_Ottawa.jpg/480px-Blue_Flag%2C_Ottawa.jpg" /> | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/590px-Iris_virginica.jpg"/> |

In [ ]:
# Download Data
import os
import urllib.request
from pathlib import Path

# Source: https://archive.ics.uci.edu/ml/datasets/iris
DATA_PATH = Path('data/1-iris.csv')
DATA_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
if not DATA_PATH.exists():
    Path('data').mkdir(exist_ok=True)
    urllib.request.urlretrieve(DATA_URL, DATA_PATH)

In [ ]:
# Read Data
# Data Structure: SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm, IrisType
import csv

data = []
with open(DATA_PATH) as fin:
    for row in csv.reader(fin):
        if not row:
            continue
        data.append((float(row[0]), float(row[1]), float(row[2]), float(row[3]), row[4]))

data[:5]

## What is the k-nearest neighbour algorithm?

Work out the nearest neighbours out by figuring out the distance to all of them, one-by-one:

![Distance between points](http://passyworldofmathematics.com/Images/pwmImagesSeven/DistanceBetweenPointsThree550x390JPG.jpg)

$$distance = \sqrt{(y_1 - x_1)^2 + (y_2 + x_2)^2}$$

Then take the `k` closest points - these are our k-nearest neighbours!

Our prediction will be the most common label in those `k` neighbours:

| Distance | Label           |
|----------|-----------------|
|   1.1    | Iris-setosa     |
|   1.5    | Iris-setosa     |
|   2.2    | Iris-versicolor |
|   2.3    | Iris-setosa     |
|   2.9    | Iris-setosa     |

In this case `Iris-setosa` wins by 4/5.

In [ ]:
import math
def get_distance(data_1, data_2):
    """Return the distance between two points."""
    return sum((x2 - x1)**2 for x1, x2 in zip(data_1[:-1], data_2[:-1]))

In [ ]:
from operator import itemgetter
from collections import Counter

def get_prediction(train_data, test_datum, knn=5, distance_func=get_distance):
    """Return our prediction for test_datum."""
    train_label_distances = []
    for train_datum in train_data:
        label = train_datum[-1]
        distance = distance_func(train_datum, test_datum)
        train_label_distances.append((label, distance))
    
    # Sort by distance (smallest to largest)
    train_label_distances.sort(key=itemgetter(1))
    # Take top k instances
    neighbours = train_label_distances[:knn]
    neighbouring_labels = [label for label, distance in neighbours]
    # Count the labels
    counted_neighbours = Counter(neighbouring_labels)
    prediction = counted_neighbours.most_common(1)[0][0]
    return prediction

## How do we test our algorithm?

We will test our algorithm against the data that we already have - rather than going out into a forest to start measuring petal lengths.

We can split the data up into training data and test data. There's a lot of ways to split it up. To start off with we will just split it up 50-50.

If we take a look at our data we can see that it has all the Iris Setosas first, then all the Veriscolors, then all the Virginicas. If we split this up 50-50, we wouldn't have any Virginicas in our training data - so it would be impossible to make a Virginica prediction!

**Randomise the data**.

In [ ]:
import random

def get_train_test_data(data):
    """Return (train_data, test_data)."""
    # Shuffle data for some variation between runs
    random.shuffle(data)
    split_line = len(data) // 2
    return (data[:split_line], data[split_line:])

To get the accuracy of our predictions, we can count the number of correct and incorrect predictions for the test data.

In [ ]:
def get_accuracy(train_data, test_data, knn):
    correct_count = 0
    for test_datum in test_data:
        prediction = get_prediction(train_data, test_datum, knn=knn)
        if prediction == test_datum[-1]:
            correct_count += 1
    
    return (correct_count / len(test_data))

def get_50_50_accuracy(data, knn=5):
    train_data, test_data = get_train_test_data(data)
    return get_accuracy(train_data, test_data, knn=5)

get_50_50_accuracy(data)

## How do we make use of all of our data?
If we run this algorithm we get a range of scores.

In [ ]:
accuracies = [get_50_50_accuracy(data) for _ in range(500)]
def get_bins(alist):
    def frange(x, y, jump):
        while x < y:
            yield x
            x += jump
    return list(frange(min(alist), max(alist), 0.01))
sns.distplot(accuracies, bins=get_bins(accuracies))

We can improve our estimate on the accuracy by using **k-fold cross-validation**.

Normally, cross-validation is used so that the algorithm can train using all of the data, while still having test (validation) data to prevent **over-fitting**.

![k-fold cross validation](https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg)

In [ ]:
def get_kfolds(data, kf):
    """Yield k (train_data, test_data)."""
    # Shuffle data for some variation between k-fold runs
    random.shuffle(data)
    test_data_size = len(data) // kf
    for i in range(kf):
        start_test_data = i * test_data_size
        end_test_data = start_test_data + test_data_size
        train_data_before = data[:start_test_data]
        test_data = data[start_test_data:end_test_data]
        train_data_after = data[end_test_data:]
        train_data = train_data_before + train_data_after
        yield (train_data, test_data)

def get_kfold_accuracy(data, kf=5, knn=12):
    accuracies = []
    for train_data, test_data in get_kfolds(data, kf=kf):
        accuracies.append(get_accuracy(train_data, test_data, knn=knn))
    return sum(accuracies) / len(accuracies)

get_kfold_accuracy(data)

If we take a look at the accuracies splitting the data into 2 chunks:

In [ ]:
accuracies = [get_kfold_accuracy(data, kf=2) for _ in range(500)]
sns.distplot(accuracies, bins=get_bins(accuracies))

## How does the number of neighbours affect things?

* Data tends to be noisy! People could have mis-measured the flowers, written down the data incorrectly, or entered the data on the computer incorrectly. Increasing the number of neighbours can mean that one flower that is so obviously wrong is ignored.
* But if you increase `k` too much, then you are comparing against *all* of the flowers... it stops being nearest neighbour!

In [ ]:
import seaborn as sns

accuracies = []
for knn in range(1, len(data)):
    accuracies.append(get_kfold_accuracy(data, knn=knn))
    
# Screw you, tsplot is the only thing that works.
sns.tsplot(accuracies, value='accuracy')

## How does the number of k-folds affect things?

* The more k-folds you have, the more you are averaging, so it should become more stable.
* The more k-folds you have, the greater the training data - reducing the chances to validate.

In [ ]:
accuracies = []
for kf in range(2, len(data)):
    accuracies.append(get_kfold_accuracy(data, kf=kf) for i in range(100))

# Screw you, tsplot is the only thing that works.
sns.tsplot(accuracies, value='accuracy')

## Is it just me, or did that take a while to run?

I agree with you Q, that was pretty slow! The performance gets worse with more k-folds k=149, 149*1*149 vs k=2, 75*75*2 ( tr-size * te-size * kf )

tr-size * te-size * kf
(1 - te-size) * (data / kf ) * kf
data * (1 - data / kf )
data - data*2/kf

As kf goes up, this goes down.

Let's time-it!

In [ ]:
%timeit get_kfold_accuracy(data)

The distance function can be optimised.

* Don't take a sqrt - only the relative distances matter.
* Use octrees/r-trees/k-d trees/etc. to subdivide objects into regions.

This can turn `O(n)` for classifying a single instance to something closer to `O(log n)`.

## Looking at the distance function

### Higher dimensions
Things get tough here.

It's pretttyyyy bad when you have lots of features.
Here's an example - let's use only Petal Widths.

In [ ]:
data_sls = [[datum[2], datum[-1]] for datum in data]
get_kfold_accuracy(data_sls)

In [ ]:
data_sls_overboard = [[datum[2]] * 20 + [datum[-1]] for datum in data]
get_kfold_accuracy(data_sls_overboard)

### Different feature "lengths"

If the features were measured in mm, cm and km you wouldn't expected the algorithm to do differently - it's the same data after all!

But it does.

In [ ]:
data_multiplied = [
    [datum[0]*10**6, datum[1]*10**1, datum[2]*10**1, datum[3]*10**1, datum[4]]
    for datum in data
]
get_kfold_accuracy(data_multiplied)

In [ ]:
import statistics
def normalise(data):
    new_cols = []
    for col_num in range(len(data[0]) -1):
        data_column = [datum[col_num] for datum in data]
        mean = statistics.mean(data_column)
        std = statistics.stdev(data_column)
        new_cols.append([(point - mean) / std for point in data_column])
    new_cols.append(datum[-1] for datum in data)
    return list(zip(*new_cols))

data_normalised = normalise(data)
get_kfold_accuracy(data_normalised)